#Lab.04 / IBM3202 – Comparative Modeling using MODELLER

##Theoretical Aspects

As we discussed during our Lectures, **the structure space is rather small when compared with the sequence space**. In fact, hierarchical structure classifications such as **CATH** have demonstrated that only a few proteins of the enormous pool of structures being deposited each year in the Protein Data Bank are known to provide novel folds. In this regard, it seems that we have discovered almost all protein folds. Moreover, the elegant experiment by Chothia & Lesk [Chothia C & Lesk AM (1986) _EMBO J_ 5(4), 823–826] revealed that, generally, **protein structures are quite similar between proteins when their sequence identity is > 30-40%**, with some remarkable exceptions to this rule).

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_01.png'/>
<figcaption>FIGURE 1. Relationship between sequence identity and structural similarity for globular and membrane proteins.
<br>Olivella M et al (2013) <i>Bioinformatics 29(13), 1589-1592</i></figcaption></center>
</figure>

This evidence provides the perfect framework to model the structure of a given protein based on another protein (a template) for which its structure has been solved and that has sufficient sequence identity. But how does it work? Well, basically it operates by using a lot of distance constraints. 

Similarly to what we saw for coevolutionary signals in our Lectures, in which pairs of residues are conserved regardless of their sequence separation because of their role in stabilizing the native state through interactions in the 3D space, we can add interaction constraints to a sequence of a protein with unknown structures based on what we observe on the template structure.

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_02.png'/>
<figcaption>FIGURE 2. From sequence similarity to spatial constraints for comparative modelling in MODELLER.
<br>Fiser A & Sali A (2003) <i>Methods Enzymol 374, 461-491</i></figcaption></center>
</figure>

##Overview

in this tutorial, we will pursue the template-based modeling of homologous enzymes of  _I. sakaiensis_ PET hydrolase (PETase) [Fecker T et al (2018) *Biophys J 114 (6), 1302-1312*] on the software **MODELLER** using a single-template approach (i.e. based on only one protein structure). Then, we will analyze the model and determine its quality using an online server.

#Part 0. Downloading and Installing the required software

Before we start, you must first **remember to start the hosted runtime in Google Colab**.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **MODELLER**, a famous software for template-based modelling.
- **py3Dmol**, for visualization of the template and modelled protein structures.
- **biopython**, for manipulation and retrieval of protein structures and sequences.

After several tests, the following installation instructions are the best way of setting up **Google Colab** for this laboratory session.

1. We will first install MODELLER as follows:

In [ ]:
#We will first download MODELLER 9.25
!wget https://salilab.org/modeller/9.25/modeller-9.25.tar.gz
#Then, we extract the downloaded folder containing MODELLER 9.25
!tar -zxf modeller-9.25.tar.gz
!echo "MODELLER extraction completed"
#Then, we switch onto the MODELLER folder
#with an automagic command
%cd modeller-9.25
#And we prepare a file containing the minimal setup elements
#For installing, including a license key
with open('modeller_config', 'a') as f:
  f.write("3\n")
  f.write("/content/compiled/MODELLER\n")
  f.write("MODELIRANJE\n")
!./Install < modeller_config
!echo "MODELLER set up completed"

In [ ]:
#Checking if installation was succesfull
#Creating a symbolic link
!ln -sf /content/compiled/MODELLER/bin/mod9.25 /usr/bin/
%cd /content/
#Checking if MODELLER works
!mod9.25 | awk 'NR==1{if($1=="usage:") print "MODELLER succesfully installed"; else if($1!="usage:") print "Something went wrong. Please install again"}'

2. Then, we will install biopython

In [ ]:
#Installing biopython using pip
!pip install biopython

3. Finally, we will install py3Dmol

In [ ]:
#Installing py3Dmol using pip
!pip install py3Dmol
#And importing the py3Dmol module
import py3Dmol

#Part	I – Retrieving the amino acid sequence of PETase homologs

Typically, this step starts with a **BLAST search** (or other types of database search) for sequence homologs of a given protein of interests, taking into consideration the e-values, sequence coverages and identities.

Here, we skipped the BLAST search that we performed during our previous tutorial and will just download one sequence.

1. Download the sequence with accession ID BAB86909.1 using biopython as we did in previous tutorials:

In [ ]:
import os
from pathlib import Path 
from Bio import SeqIO, Entrez
seqlist = ['BAB86909.1']
for n in seqlist:
  #Creating folder for our sequence
  if not os.path.exists(n):
    os.mkdir(n)
  folder = Path(n)
  #Setting up your email to be able to use Entrez
  Entrez.email = 'your.email@uc.cl'
  #Here, we set up a temporary handle with our downloaded sequence in fasta format
  temp = Entrez.efetch(db="protein",rettype="fasta",id=n)
  #Creating a fasta file to write our downloaded sequence
  aaseq_out = open(folder/ "target.fasta",'w')
  #Reading the sequence information as a string in fasta format
  aaseq = SeqIO.read(temp, format="fasta")
  #Writing the sequence record in fasta format
  SeqIO.write(aaseq,aaseq_out,"fasta")
  #Closing both the temp handle and the FASTA file
  temp.close()
  aaseq_out.close()

2. If everything worked smoothly, your sequence should have been downloaded as a FASTA file in an appropriate folder named after its accession ID. Could you check what is the description and sequence of this file?

In [ ]:
#Obtain the description of the downloaded protein

#Obtain the sequence of the downloaded protein


3. It is highly recommended at this point that you change the name of the sequence (labeled as “>”) to something shorter, such as just the ID or a name (e.g. target). You can simply do this manually by opening this file on Google Colab.

#Part	II – Select an appropriate template structure and perform a sequence alignment for protein structure modeling

Selecting an appropriate template for modeling a structure of a homologous protein is as crucial as an appropriate alignment to correctly position the different residues.

**QUESTION:** What features of a crystal structure do you think are important for choosing the best template?

1. As we will  work on the sequence BAB86909.1, we will first change directory.

In [ ]:
%cd BAB86909.1

2. We will find the potential best templates from the whole PDB database to model the structure of our target protein using the **profile.build()** command from MODELLER.

  For this purpose, we need a text file containing a list of non-redundant PDB sequences at 95% sequence identity and an appropriate script for running MODELLER.

In [ ]:
#Downloading pdb_95.pir
!wget https://salilab.org/modeller/downloads/pdb_95.pir.gz
!gunzip pdb_95.pir.gz
#Downloading the build_profile.py script from GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/build_profile.py

In [ ]:
#Running the build_profile script
!mod9.25 build_profile.py
#Printing only the list of potential templates
!sed -n '/HITS FOUND IN ITERATION:     1/,/Weight Matrix/p;/Weight Matrix/q' build_profile.log

In this particular example, a BLOSUM62 similarity matrix is being used for determining the sequence identity between target and potential templates. Also, we are employing only one search iteration and the parameter max_aln_evalue is set to 0.01, indicating that only sequences with e-values smaller than or equal to 0.01 will be included in the final profile.

For simplicity, we just printed out the PDB table from the resulting log file generated during this analysis.

As you can see, several PDB files are indicated. The important columns to determine the best templates from this analysis are the fifth, seventh and eight columns, which correspond to the sequence length of the target protein, the sequence identity and the e-value, respectively.

**QUESTION:** From this analysis, which template would be better for modeling the structure of our target sequence?

2. We will choose five PDB structures based on the sequence identity and e-value and select the most appropriate template for our target sequence among them. For this, we will first download these structures using the _Bio.PDB_ command from biopython, and then use the alignment.compare_structures() command to assess the structural and sequence similarity between the possible templates through the **compare.py** script.

  Please take a few minutes to examine the content of this script, particularly i) how are the different protein structures included within the script; and ii) which chain is being used from each structure. This is important for cases where only one of many chains in the PDB corresponds to the protein. Also, please note that there are two alignment steps: first, a sequence alignment; second, a structural alignment.

In [ ]:
#Downloading the PDB files using biopython
from Bio.PDB import *
templates = ['6eqe', '5luk', '6aid', '4cg1', '6sbn']
pdbl = PDBList()
for s in templates:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#Downloading the build_profile.py script from GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/compare.py
#Check this script and change the names of the PDB files if required

In [ ]:
#Running the compare script
!mod9.25 compare.py
#Check the log file
!sed -ne '/Sequence identity comparison (ID_TABLE):/,$ p' compare.log

Several decisions can be made based on the results from this analysis, which were filtered in the previous code cell for simplicity.

The resulting comparison between structures shows that PDB 6eqe has the highest resolution (0.9 Å). Secondly, it shows that 5luk and 4cg1 are very similar between them (92% sequence identity), and also similar to 6aid (82-85% sequence identity), forming a distinctive group of structures. Third, 6sbn is the structure with the least similarity between candidates. Lastly, that 6eqe is part of an isolated group.

Thus, considering the sequence identity between the target sequence and 6eqe, its high resolution and other crystallographic quality features (that you can check on the **[PDB website](https://www.rcsb.org/)**), this is the best candidate for single-template modeling. 

3. Now, we will **align the sequence of our template protein with the sequence of our target protein**, such that we can model the structure.

  How hard is it? Not at all! Just download the  **align2d.py** script into your working folder, check the script to verify how the sequence of the target and the protein structure are evoked, and execute the script as we have done before



In [ ]:
#Downloading the align2D.py script from GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/align2D.py
#Check this script and change the names of the PDB files if required

In [ ]:
#Running the align2D script
!mod9.25 align2D.py

4. You will end up with two new files (aligned.ali and aligned.fasta) that contain the pairwise alignment of the target and template sequences. Load the FASTA file onto [MSAviewer](http://msa.biojs.net/app/).

  You will see that something odd is happening: **a large segment of the N-terminal has no equivalent residues in the template structure!**

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_03.png'/>
</center>
</figure>

  This exercise leads to a general recommendation: you should always check all possible information on your biological sequences and structures to identify if these conflicts have any biological explanation. **THIS IS FUNDAMENTAL**.

5. Go to the PDB website and check the annotation of the sequence of PETase in Protein Feature View. What do you see?

  Based on your observations, decide whether you should truncate the sequence of your target before modeling, edit your target sequence and repeat the alignment, if required.


#Part III - Generate and visualize a comparative model using MODELLER

1. Once your target and template sequences are aligned, use the **model-single.py** script for finally obtaining a structure of your target through comparative modeling. Again, read the script and check how the sequences and structures are called in MODELLER through these scripts. In this case, we are also performing this step on a separate folder.

  Please note that 1 model is not enough, as there is an energy function defining the optimal position of atomic coordinates, thus different models will have different energies. Generally, between 50-100 are generated for sufficient evaluation.

**💡 HINT:** The generation of 50 models takes around 15 min on Google Colab.

In [ ]:
#Creating a new folder and copying the required files for MODELLER
%mkdir model-single
%cd model-single
%cp ../6eqe.pdb .
%cp ../aligned.ali .

In [ ]:
#Downloading the model-single.py script from GitHub
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/scripts/model-single.py
#Check this script and change the names of the PDB files if required

In [ ]:
#Running the model-single script
!mod9.25 model-single.py

2. The output from this process is a bunch of PDB files, each one of them corresponding to a comparative model of our target protein, that are numbered from 1 up to the total number of models requested during comparative modeling.

  Also, the **model-single.log** output has the total potential energy for each structure,according to MODELLER’s DOPE (discrete optimized protein energy) score. For simplicity, this script was modified to indicate the model with the best DOPE score. We will be working only with the model with the best score for the remainder of the session.
  
  As an example, our best model during preparation of this tutorial showed the following DOPE score:

```
Top model: target.B99990025.pdb (DOPE score -28735.180)
```

3. Before we check the quality of our model, we will take a look at it on py3Dmol. Do you remember how to do this? We also included a small trick to generate an image!

In [ ]:
view=py3Dmol.view()
#Loading template
view.addModel(open('6eqe.pdb', 'r').read(),'pdb')
#Loading best DOPE score model
view.addModel(open('YOURMODEL', 'r').read(),'pdb')
view.zoomTo()
view.setBackgroundColor('white')
view.setStyle({'chain':'A'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':' '},{'cartoon': {'color':'yellow'}})
view.show()
#The following is a small trick to be able to manipulate (rotate, zoom) your
#representation in Google Colab and, once you are happy with the visualization,
#press Enter to print out as an onscreen png image file that you can download
input()
view.png()

4. Finally, to check the stereochemical quality of the model and its comparison to experimentally solved structures, we will use the [SAVES server](http://servicesn.mbi.ucla.edu/SAVES/), which employs several structure-based scoring strategies:

* **VERIFY3D** (i.e. compatibility of an atomic 3D model to its 1D sequence when compared tothe energetics of good structures from the PDB).
* **ERRAT** (i.e. quality of non-bonded interactions of a region when compared to similar regions from highly refined structures).
* **PROCHECK** (stereochemical and geometrical quality of the model, via Ramachandran plots, sidechain rotamers, etc).

5. Download your best model, upload it to SAVES and wait for the results. Briefly:
- **Check the VERIFY3D results:** >80% of the residues should have an average score ≥ 0.2, whereas the score profile allows you to identify conflicting regions.
- **Check the Ramachandran plot:** Are there any residues outside the allowed regions? What types of residues are found within those regions? (Check it by clicking on each dot in the plot)
- **Check the errors in PROCHECK:** are the errors located within the loop regions?


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/cm_04.png'/>
</center>
</figure>

While we will discuss some of these results at the end of this tutorial, we highly encourage you to read [this article](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007449), which contains more recommendations on comparative modelling of protein structures.

**This is the end of the fourth tutorial! Good Science!!**